# Lab 9-5: MNIST NN Dropout
Dropout은 overfitting(과적합) 문제를 해결하기 위한 한 가지 방법으로, hidden layer(은닉층)의 일부 유닛을 비활성화 하여 학습하는 방법이다.

In [1]:
#import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [2]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform= transforms.ToTensor(),
                         download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform= transforms.ToTensor(),
                         download= True)

#dataset loader
data_loader = torch.utils.data.DataLoader(dataset= mnist_train,
                                          batch_size= batch_size,
                                          shuffle=True,
                                          drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [3]:
#nn layers
linear1 = torch.nn.Linear(784, 512, bias= True)
linear2 = torch.nn.Linear(512, 512, bias= True)
linear3 = torch.nn.Linear(512, 512, bias= True)
linear4 = torch.nn.Linear(512, 512, bias= True)
linear5 = torch.nn.Linear(512, 10, bias= True)
relu = torch.nn.ReLU()

dropout = torch.nn.Dropout(p= drop_prob)

#xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

#model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) #Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [4]:
#train
total_batch = len(data_loader)
model.train() #set the model to train mode (dropout = True)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print('Epoch: {:2d} Cost= {:.9f}'.format(epoch + 1, avg_cost))
print('Learning finished')

Epoch:  1 Cost= 0.310621142
Epoch:  2 Cost= 0.146159276
Epoch:  3 Cost= 0.111172289
Epoch:  4 Cost= 0.094859935
Epoch:  5 Cost= 0.082409069
Epoch:  6 Cost= 0.072757378
Epoch:  7 Cost= 0.071835004
Epoch:  8 Cost= 0.059738025
Epoch:  9 Cost= 0.059085678
Epoch: 10 Cost= 0.055539340
Epoch: 11 Cost= 0.051415320
Epoch: 12 Cost= 0.047818515
Epoch: 13 Cost= 0.045675199
Epoch: 14 Cost= 0.046652373
Epoch: 15 Cost= 0.046102792
Learning finished


In [5]:
#test
with torch.no_grad():
  model.eval() #set the model to elvaluation mode (dropout = False)
  
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.980400025844574
Label: 8
Prediction: 8
